<a href="https://colab.research.google.com/github/anandita-garg/Predicting-Optimal-Locations-For-Solar-Farms/blob/main/AHP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

df = pd.read_csv("test.csv")

df.rename(columns={
    'latitude': 'Latitude',
    'longitude': 'Longitude',
    'suitability_score_1_x': 'AHP1',
    'suitability_score_2_x': 'AHP2',
    'ahp_score_x': 'Suitability Score',
    'lulc_code': 'LULC/Land Cover',
    'elevation': 'Elevation',
    'slope': 'Slope',
    'cloud_cover': 'Cloud Cover',
    'cloudy_days_percent': 'Cloud Cover',
    'humidity': 'Humidity',
    'temperature': 'Temperature',
    'solar_radiation': 'Solar Radiation',
    'ndvi': 'Vegetation',
    'aspect class': 'Aspect',
    'wind speed (m/s)': 'Wind Speed',
    'lst (°c)': 'LST',
    'distance_to_road_combined': 'Distance From Roads',
}, inplace=True)

df.drop(columns=[col for col in ['lulc_type', 'Vegetation Class', 'Aspect Direction'] if col in df.columns], inplace=True)

df.head()
count = (df['AHP1'] == 1).sum()
print(f"Number of rows with Suitability Score = 1: {count}")


Number of rows with Suitability Score = 1: 539


In [ ]:
numeric_cols = df.select_dtypes(include=[np.number]).columns.difference(['Latitude', 'Longitude', 'AHP1', 'AHP2', 'Suitability Score'])

for col in numeric_cols:
    df[col] = (df[col] - df[col].min()) / (df[col].max() - df[col].min())


df['Cloud Cover'] = 1 - df['Cloud Cover']
df['Humidity'] = 1 - df['Humidity']
df['Distance From Roads'] = 1 - df['Distance From Roads']
df['LULC/Land Cover'] = 1 - df['LULC/Land Cover']
df['Temperature'] = 1 - df['Temperature']
df['Slope'] = 1 - df['Slope']
df['LST'] = 1 - df['LST']

In [ ]:
def compute_suitability_score1(df):
    weights = {
        'Solar Radiation': 10,
        'Temperature': 15,
        'Cloud Cover': 5,
        'Humidity': 5,
        'Elevation': 12,
        'Slope': 35,
        'LULC/Land Cover': 10,
        'Distance From Roads': 8
    }

    missing = [col for col in weights if col not in df.columns]
    if missing:
        raise ValueError(f"Missing columns in DataFrame: {missing}")

    weights_series = pd.Series(weights, index=df[list(weights)].columns)

    if 'AHP1' not in df.columns:
        df['AHP1'] = pd.NA

    mask = df['AHP1'] != 1

    weighted_scores = df.loc[mask, list(weights.keys())].mul(weights_series, axis=1).sum(axis=1)
    normalized_scores = (weighted_scores - weighted_scores.min()) / (weighted_scores.max() - weighted_scores.min())

    update_mask = df['AHP1'].isna() | (df['AHP1'] == 0)
    final_mask = mask & update_mask
    df.loc[final_mask, 'AHP1'] = normalized_scores[final_mask]

    return df

compute_suitability_score1(df)
count = (df['AHP1'] == 1).sum()
print(f"Number of rows with Suitability Score = 1: {count}")


Number of rows with Suitability Score = 1: 540


In [ ]:
def compute_suitability_score2(df):
    weights = {
        'Solar Radiation': 13.56,
        'Wind Speed': 12.71,
        'LST': 11.86,
        'Humidity': 11.02,
        'Vegetation': 10.17,
        'LULC/Land Cover': 9.32,
        'Distance From Roads': 9.32,
        'Elevation': 11.02,
        'Aspect': 11.02,
    }

    missing = [col for col in weights if col not in df.columns]
    if missing:
        raise ValueError(f"Missing columns in DataFrame: {missing}")

    weights_series = pd.Series(weights)

    if 'AHP2' not in df.columns:
        df['AHP2'] = pd.NA

    mask = df['AHP2'] != 1

    weighted_scores = df.loc[mask, list(weights.keys())].mul(weights_series, axis=1).sum(axis=1)
    normalized_scores = (weighted_scores - weighted_scores.min()) / (weighted_scores.max() - weighted_scores.min())

    update_mask = df['AHP2'].isna() | (df['AHP2'] == 0)
    final_mask = mask & update_mask
    df.loc[final_mask, 'AHP2'] = normalized_scores[final_mask]

    return df



compute_suitability_score2(df)

def compute_final_suitability_score(df):
    suitability_scores = []

    for _, row in df.iterrows():
        if row['AHP1'] < 0.2 or row['AHP2'] < 0.2:
            suitability_scores.append(min(row['AHP1'], row['AHP2']))
        else:
            suitability_scores.append((row['AHP1'] + row['AHP2']) / 2)

    df['Suitability Score'] = suitability_scores

    return df

compute_final_suitability_score(df)

,Latitude,Longitude,AHP1,AHP2,Suitability Score,LULC/Land Cover,Elevation,Slope,Cloud Cover,Cloud Cover,...,total sunshine (min),total snowfall (mm),max surface pressure (hpa),max dew point (°c),total shortwave radiation (wh/m²),total reference evapotranspiration (mm),pm2.5,no2,so2,dust
0,23.373390,83.171633,0.559409,0.409736,0.484572,0.50,0.088023,0.971831,0.985957,0.823799,...,0.874344,0.0,0.881104,0.814286,0.755645,0.710988,0.386122,0.166294,0.131711,0.010122
1,13.394444,79.087583,0.585932,0.340021,0.462976,0.75,0.054994,0.985915,0.984717,0.809357,...,0.816102,0.0,0.920361,0.831429,0.772620,0.764197,0.252010,0.185406,0.051382,0.002043
2,23.892835,83.045389,0.500282,0.258959,0.379621,0.50,0.072471,0.929577,0.985957,0.823799,...,0.870253,0.0,0.902893,0.840000,0.748925,0.740134,0.423346,0.305038,0.214042,0.014360
3,22.722701,75.853760,0.507372,0.183226,0.183226,0.00,0.088504,1.000000,0.987568,0.858462,...,0.858543,0.0,0.881292,0.800000,0.751597,0.799924,0.337484,0.197568,0.075421,0.011718
4,25.598578,84.461164,0.557458,0.411678,0.484568,0.75,0.009780,0.971831,0.987801,0.864239,...,0.865175,0.0,0.988355,0.897143,0.700390,0.730200,0.761868,0.320179,0.139924,0.014822
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5930,26.548458,85.686765,0.450020,0.266046,0.358033,0.75,0.010261,0.985915,0.029435,0.434208,...,0.858989,0.0,0.987603,0.882857,0.670123,0.635516,0.797147,0.327873,0.137720,0.009682
5931,29.875100,75.320346,0.458116,0.335044,0.396580,0.75,0.034472,0.915493,0.041180,0.585238,...,0.887925,0.0,0.954545,0.885714,0.711318,0.731059,0.790532,0.387689,0.109776,0.119546
5932,20.274721,81.959227,0.492407,0.438257,0.465332,1.00,0.073593,0.929577,0.029600,0.424769,...,0.813436,0.0,0.899887,0.817143,0.736743,0.718258,0.339559,0.064780,0.088141,0.004123
5933,22.912042,71.680284,0.484083,0.378273,0.431178,0.75,0.008818,0.971831,0.043199,0.594677,...,0.880544,0.0,0.984035,0.862857,0.771792,0.872017,0.234501,0.115910,0.050881,0.020577


In [ ]:
df.to_csv("test.csv", index=False)